<a href="https://colab.research.google.com/github/GuillermoSainz07/Notebooks-DS/blob/main/clases_admin_riesgos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Clases creadas para calcular VaR en activos y portafolios de inversion

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import yfinance as yf

In [ ]:
class VaR:

    def __init__(self, start=None, end=None, ticker=None, interval='1d'):

        self.start = start
        self.end = end
        self.ticker = ticker
        self.interval = interval
        try:
            data = yf.download(tickers=self.ticker,
                              start=self.start,
                              end=self.end,
                              interval=self.interval,
                              progress=False).Close
        except:
            print('Asegurate de agregar fecha de inicio, fin y ticker')

        data = pd.DataFrame(data)
        data['rendimientos'] = data.pct_change()
        data.dropna(inplace=True)

        self.prices = data.Close
        self.rendimientos = data.rendimientos

    def compute_var(self, var = 95):
        self.var = var
        self.percentile = 100-self.var
        self.var_computed = np.percentile(self.rendimientos, self.percentile)
        return self.var_computed

    def plot_histogram(self, kde=True, color='purple'):
        if kde:
            sns.histplot(self.rendimientos, stat='density', alpha=0.5, label='Histogram')
            sns.kdeplot(self.rendimientos, color=color, label='density')
            plt.axvline(self.var_computed, ls='--', color='red', label=f'VaR {self.var}%')
            plt.title(f'Histograma de los rendientos {self.ticker}')
            plt.legend()

        else:
            sns.histplot(self.rendimientos,label='Histogram')
            plt.axvline(self.var_computed, ls='--', color='red', label=f'VaR {self.var}%')
            plt.title(f'Histograma de los rendimientos {self.ticker}')
            plt.legend()

    def plot_prices(self):
        self.prices.plot(title=f'Prices Time Series {self.ticker}')

    def plot_returns(self):
        self.rendimientos.plot(title=f'Returns Time Series {self.ticker}')

In [ ]:
class VaR_Portfolio:

    def __init__(self,tickers=None, interval='1d',df=None):

        self.tickers = tickers
        self.interval = interval
        try:
            data = yf.download(tickers=self.tickers,
                              interval=self.interval,
                               start='2022-08-30',
                               end='2023-08-30',
                              progress=False)[['Adj Close']]
            list_tickers = tickers
            list_names = ['retornos' for _ in range(len(list_tickers))]
            list_tuplas = list(zip(list_names, list_tickers))

            self.data = data
            self.data[list_tuplas] = self.data.pct_change()

            self.data.dropna(inplace=True)

            self.prices = self.data['Adj Close']
            self.rendimientos = self.data.retornos
        except:
            data = df

            self.tickers = df.columns.tolist()

            self.data = data
            self.prices = self.data

            self.rendimientos = self.data.pct_change().dropna()

    def compute_var(self,units, var=0.01):
        self.units = units
        assert len(units) == len(self.tickers), 'Agrega las unidades para cada activo'

        self.amount = self.prices.iloc[-1].values * units
        self.weight = self.amount / np.sum(self.amount)

        self.ret_port = self.rendimientos @ self.weight.reshape(-1,1)
        self.ret_port.columns = ['ret_port']

        self.var_port = {'%':self.ret_port.quantile(var)[0],
                        '$': self.ret_port.quantile(var)[0] * np.sum(self.amount)}
        self.var_df = pd.DataFrame(self.var_port, index=['var'])

        self.cvar_port = {'%': np.mean(self.ret_port[self.ret_port < self.var_port['%']]),
                         '$': np.mean(self.ret_port[self.ret_port < self.var_port['%']]) * np.sum(self.amount)}

        self.cvar_df = pd.DataFrame(self.cvar_port, index=['cvar'])

        self.cov = self.rendimientos.cov()
        self.variance_portfolio = self.weight.T @ self.cov @ self.weight
        self.vol_port = np.sqrt(self.variance_portfolio)

    def stress_test(self,sigmas_n=5):

        self.stress_sigma = {'sigmas': -sigmas_n,
                      '$': (-sigmas_n *self.vol_port)* np.sum(self.amount)}

        self.stress_sigma_df = pd.DataFrame(self.stress_sigma,index=['stress_sigma'])

        self.low = self.ret_port.min()
        self.sigma_history = self.low / self.vol_port
        self.stress_history = {'sigmas': self.sigma_history[0],
                       '$': self.sigma_history[0] * np.sum(self.amount)}
        self.stress_history_df = pd.DataFrame(self.stress_history, index=['stress_history'])


    def plot_prices(self, normalize=False):
        scaler = StandardScaler()
        norm_price = scaler.fit_transform(self.prices)

        if normalize:
            plt.plot(norm_price)
            plt.title('Precios estandarizados')

        else:
            self.prices.plot(title='Precios normales')

    def plot_portafolio(self):
        rg = (((1 + self.ret_port).prod())**(1/len(self.ret_port)) - 1 )[0]
        vf = np.sum(self.amount)
        v0 = vf /((1 + rg)**len(self.ret_port))
        value_port = [np.sum(self.units * self.prices.iloc[i]) for i in range(len(self.prices))]
        mean_port_perf = sorted([vf /((1 + rg)**i) for i in range(1,len(self.ret_port)+1)], reverse=False)

        fig, ax = plt.subplots(1,2, figsize=(12,6))
        ax[0].plot(self.rendimientos.index, value_port)
        ax[0].set_title('Portfolio performance')

        ax[1].plot(self.rendimientos.index, mean_port_perf)
        ax[1].set_title('Mean portfolio performance')

In [ ]:
class VaR_horizonte:

    def __init__(self,ticker=None, interval='1d', df=None):

        self.ticker = ticker
        self.interval = interval
        try:
            data = yf.download(tickers=self.ticker,
                              interval=self.interval,
                              progress=False)['Adj Close']

            self.data = pd.DataFrame(data)
            self.data['rendimientos'] = self.data.pct_change()
            self.data.dropna(inplace=True)

            self.prices = self.data['Adj Close']
            self.rendimientos = self.data.rendimientos

        except:
            self.data = df

            self.data['rendimientos'] = df.pct_change()
            self.data.dropna(inplace=True)

            self.prices = df
            self.rendimientos = self.data.rendimientos


    def window_years(self, year=10, var=0.01,date='2020-01-01'):
        self.var = var
        self.date = date
        self.year = year
        mount_data = int(252*self.year)

        self.amount = mount_data

        var_rolling = self.data[['rendimientos']].rolling(window=self.amount)\
        .apply(lambda df: df.quantile(self.var))

        var_rolling.dropna(inplace=True)

        self.var_rolling = var_rolling[var_rolling.index > self.date]

        self.rendimientos = self.data[['rendimientos']]\
        [self.data[['rendimientos']].index > self.date]

        self.outlier_var = self.rendimientos[self.rendimientos < self.var_rolling].dropna()
        self.outlier_var_short = self.rendimientos[self.rendimientos > self.var_rolling].dropna()
        self.index = self.rendimientos[self.rendimientos < self.var_rolling].dropna().index
        self.index_short = self.rendimientos[self.rendimientos > self.var_rolling].dropna().index

        self.cvar_long = np.mean(self.outlier_var)
        self.cvar_short = np.mean(self.outlier_var_short)

    def plot_window(self, outlier=True, outlier_short=False):
        plt.plot(self.rendimientos)
        plt.plot(self.var_rolling, label='VaR')
        if outlier:
            plt.scatter(self.index, self.outlier_var, color='red')

        elif outlier_short:
             plt.scatter(self.index_short, self.outlier_var_short, color='red')
        else:
            pass

In [ ]:
class Tasa_Port:
    def __init__(self,precios):

        self.dv01 = {f'bono_{i}':val[0]-val[1] for i,val in enumerate(precios)}


    def compute_port(self,data,titulos):


        assert len(titulos) == len(self.dv01), 'Ingresa la misma cantidad de titulos que de bonos'

        data2 = data.copy()
        self.columns = data2.columns.tolist()

        df_pl = pd.DataFrame()

        for p, column in enumerate(self.columns):

            data[f'{column}_pb'] = data[column].diff()*100
            data[f'{column}_pl'] = data[f'{column}_pb']*self.dv01[f'bono_{p}']*titulos[p]*-1
            df_pl[f'{column}_pl'] = data[f'{column}_pb']*self.dv01[f'bono_{p}']*titulos[p]*-1

        data['pl_port'] = df_pl.sum(axis=1)

        self.data = data

        self.var = self.data[['pl_port']].quantile(0.025)
        self.cvar = self.data[['pl_port']][self.data.pl_port < self.var[0]].mean()